# Bienvenido a la notebook de implementación de ...

## 28 EMOCIONES

## joeddav/distilbert-base-uncased-go-emotions-student

In [37]:
import pickle
import numpy as np
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
import torch

In [38]:
workdir = os.getcwd()

file_extension = ".pickle"
upload_folder = os.path.join(workdir, "output_transcripcion", "sentiment-analysis")

# bloques descargados
bloques = os.listdir(upload_folder)

debate_dict = {}
for bloque in bloques:
    debate_dict[bloque] = {}
    upload_folder_bloque = os.path.join(upload_folder, bloque)
    # # filter files by extension
    files = os.listdir(upload_folder_bloque)
    files = [f for f in files if f.endswith(file_extension)]

    for file in files:
        with open(os.path.join(upload_folder_bloque, file), 'rb') as f:
            # file remove extension
            file = file[:-len(file_extension)]
            debate_dict[bloque][file] = pickle.load(f)


In [39]:
## [ORIGINAL] : este funciona como un clasificador de una sola etiqueta

if False:

    MODEL_NAME = "joeddav/distilbert-base-uncased-go-emotions-student"
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    try:
        num_gpus = torch.cuda.device_count()
        for ix in range(0, num_gpus):
            # select the latest recognized gpu - the only one in this case
            device_id = 'cuda:{}'.format(ix)
            device = torch.device(device_id)
            print("GPU disponible, otorgada mediante el id: {}".format(device_id))
    except:
        print("Esto debería imprimirse si no existen GPUs disponibles")
    
    
    # create instances of tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(device)
    
    
    # tokenize
    tokens = tokenizer(muestra, return_tensors="pt", padding=True, truncation=True, max_length=512)
    # predict
    # Predice
    with torch.no_grad():
        tokens = {key: value.to(device) for key, value in tokens.items()}  # Mueve los tensores a la GPU (o CPU)
        outputs = model(**tokens)
    

In [40]:
MODEL_NAME = "joeddav/distilbert-base-uncased-go-emotions-student"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

try:
    num_gpus = torch.cuda.device_count()
    for ix in range(0, num_gpus):
        # select the latest recognized gpu - the only one in this case
        device_id = 'cuda:{}'.format(ix)
        device = torch.device(device_id)
        print("GPU disponible, otorgada mediante el id: {}".format(device_id))
except:
    print("Esto debería imprimirse si no existen GPUs disponibles")


# create instances of tokenizer and model
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=device, top_k=None)


GPU disponible, otorgada mediante el id: cuda:0


In [41]:
debate_dict.keys()

dict_keys(['bloque_de_desarrollo_humano_vivienda_y_protección_del_ambiente', 'bloque_de_preguntas_cruzadas', 'bloque_de_seguridad', 'bloque_de_trabajo_y_producción', 'introduccion'])

In [42]:
bloque = 'bloque_de_trabajo_y_producción'

prediccion_dict = {}
for participacion in debate_dict[bloque].keys():
    prediccion_dict[participacion] = []
    corpus = debate_dict[bloque][participacion]

    with torch.no_grad():
        outputs = pipeline(corpus)

    for ix, output in enumerate(outputs):
        prediccion_dict[participacion].append(output)

c:\Users\Agustin\Desktop\RD\Octubre\MODELO-16-EMOTIONS\venv\Lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [43]:
## Creacion de un único DataFrame con todos los scores de cada línea de su discurso.
from itertools import count

predicciones_frame = {}
for participante in prediccion_dict.keys():
    N = count(1)
    df_limpio = pd.DataFrame()
    for participacion in prediccion_dict[participante]:
        df_aux = pd.DataFrame(participacion).set_index("label")
        df_aux *= 100
        df_aux["score"] = df_aux["score"].round(2)
        participante_N = participante.split("_")[1] + "_" + str(next(N))#.zfill(2)#TODO: eliminar de j.r.d
        print(participante_N)
        df_aux = df_aux.rename(columns={"score": participante_N})
        df_limpio = pd.concat([df_limpio, df_aux], axis=1)
    predicciones_frame[participante] = df_limpio
        
    
## Ejemplo de como comparar los scores con una linea del discurso en particular
# si analizamos Bullrich_8, vemos que el score da:
#   - 7,7% aprobación
#   - 3.5; 3.4, 3., 3.4 de amor, optimismo, admiracion, alegría, aproximadamente
#   - luego analizamos la línea en particular y vemos que dice:
#           " Y Milei se ha asociado hace pocos días con ellos también "
# linea = 8
# predicciones_frame["2_Bullrich_1"]
# pd.DataFrame(debate_dict["introduccion"]["2_Bullrich_1"]).T.iloc[:,linea-1].values[0]# Transmite Alegría, Amor, 7,7% aprobación, etc...

# fin de la demostración

Schiaretti_1
Schiaretti_2
Schiaretti_3
Schiaretti_4
Schiaretti_5
Schiaretti_6
Schiaretti_7
Schiaretti_8
Schiaretti_9
Schiaretti_10
Schiaretti_11
Schiaretti_12
Massa_1
Massa_2
Massa_3
Massa_4
Massa_5
Massa_6
Massa_7
Massa_8
Massa_9
Massa_10
Massa_11
Massa_12
Massa_13
Massa_14
Bregman_1
Bregman_2
Bregman_3
Bregman_4
Bregman_5
Bregman_6
Milei_1
Milei_2
Milei_3
Milei_4
Milei_5
Milei_6
Milei_7
Schiaretti_1
Schiaretti_2
Schiaretti_3
Schiaretti_4
Schiaretti_5
Schiaretti_6
Schiaretti_7
Bullrich_1
Bullrich_2
Bullrich_3
Bullrich_4
Bullrich_5
Bullrich_6
Massa_1
Massa_2
Massa_3
Milei_1
Milei_2
Milei_3
Schiaretti_1
Schiaretti_2
Schiaretti_3
Schiaretti_4
Massa_1
Massa_2
Massa_3
Massa_4
Schiaretti_1
Schiaretti_2
Bregman_1
Bregman_2
Bregman_3
Bregman_4
Bregman_5
Bregman_6
Bregman_7
Bregman_8
Bregman_9
Bregman_10
Milei_1
Milei_2
Milei_3
Milei_4
Milei_5
Milei_6
Milei_7
Milei_8
Milei_9
Milei_10
Bregman_1
Bregman_2
Bregman_3
Milei_1
Milei_2
Milei_3
Milei_1
Milei_2
Milei_3
Milei_4
Milei_5
Milei_6
Milei_7
M

## Exportar los dataframe

In [44]:
from tools.feed import crear_directorio
for frame in predicciones_frame.keys():# estamos usando variables de otra celda - bloque
    output_folder = os.path.join(workdir, "output_model_emotions", bloque)
    crear_directorio(output_folder, verbose=False)
    
    participacion = pd.DataFrame(debate_dict[bloque][frame]).T
    
    frame_num = frame.split("_")[0].zfill(2)
    frame_name = frame_num + "_" + "_".join(frame.split("_")[1:])##TODO: actualizar en j.r.d
    
    
    participacion.to_csv(os.path.join(output_folder, frame_name + "_humano.csv"))
    with open(os.path.join(output_folder, frame_name + "_prediccion.pickle"), 'wb') as f:
        pickle.dump(predicciones_frame[frame], f)
        

In [45]:
participacion

,0,1,2,3,4,5,6,7,8,9,10
0,"La Argentina tiene todo, pero los argentinos n...",Hace 20 años que la mafia política y sindical ...,"Un juicio laboral se lleva a puesta una pyme, ...",Un empleo solo se consigue en la informalidad,Por eso nosotros queremos sacarle la pata enci...,"Y nosotros defendemos a los que trabajan, no a...",Ustedes defienden a los sindicalistas que está...,Nosotros vamos a bajar impuestos al trabajo,Nosotros queremos hacer una verdadera ley para...,"Por otro lado, te lo digo clarito, las indemni...",Cambia la Argentina
